In [26]:
import numpy as np
import random
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import nltk

nltk.download("punkt")
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
# 2. NLTK Utilities

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    return bag


In [29]:
# 3. Model Definition

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = self.l3(out)
        return out


In [33]:
# 4. Load Intents Data

with open("intents.json", "r") as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    for pattern in intent["patterns"]:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ["?", ".", "!"]
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns", xy)
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words", all_words)


38 patterns [(['Hi'], 'greeting'), (['Hello'], 'greeting'), (['Hey'], 'greeting'), (['Good', 'morning'], 'greeting'), (['Good', 'evening'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['How', 'can', 'I', 'apply', 'for', 'admission', '?'], 'admission_process'), (['What', 'is', 'the', 'admission', 'process', '?'], 'admission_process'), (['How', 'do', 'I', 'get', 'admission', 'in', 'Gujarat', 'Vidyapith', '?'], 'admission_process'), (['Tell', 'me', 'about', 'the', 'admission', 'procedure'], 'admission_process'), (['Which', 'courses', 'are', 'available', '?'], 'courses'), (['What', 'programs', 'do', 'you', 'offer', '?'], 'courses'), (['List', 'of', 'courses'], 'courses'), (['Tell', 'me', 'about', 'the', 'courses', 'in', 'Gujarat', 'Vidyapith'], 'courses'), (['What', 'is', 'the', 'eligibility', 'for', 'admission', '?'], 'eligibility'), (['Who', 'can', 'apply', '?'], 'eligibility'), (['Admission', 'requirements'], 'eligibility'), (['Minimum', 'qualification', 'needed'], 'elig

In [34]:
# 5. Prepare Training Data

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(f"Input size: {input_size}, Output size: {output_size}")

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples


Input size: 78, Output size: 9


In [35]:
# 6. Training

batch_size = 8
learning_rate = 0.001
num_epochs = 500

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNet(input_size, hidden_size, output_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        outputs = model(words)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print(f"Final loss: {loss.item():.4f}")

# Save model state in memory
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)
print(f"Training complete. File saved to {FILE}")


Epoch [100/500], Loss: 0.9855
Epoch [200/500], Loss: 0.1024
Epoch [300/500], Loss: 0.0164
Epoch [400/500], Loss: 0.0070
Epoch [500/500], Loss: 0.0039
Final loss: 0.0039
Training complete. File saved to data.pth


In [36]:
# 7. Chat Function

data = torch.load(FILE)
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "UniBuddy"

def chat_with_bot(sentence):
    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]
    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.75:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                return random.choice(intent["responses"])
    else:
        return "I do not understand..."
